<a href="https://colab.research.google.com/github/Sushmithaa-Pandian/Machine_learning_sush/blob/main/Pnem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [14]:
IMAGE_SIZE = [224, 224]



In [15]:
net = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [16]:
for layer in net.layers:
    layer.trainable = False

In [17]:
train_path = '/content/gdrive/MyDrive/pnem1/train'
valid_path = '/content/gdrive/MyDrive/pnem1/test'

In [18]:
folders = glob('/content/drive/MyDrive/pnem1/train/*')

In [19]:
x = Flatten()(net.output)
prediction = Dense(units=1, activation='sigmoid')(x)

# create a model object
model = Model(inputs=net.input, outputs=prediction)
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [20]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/gdrive/MyDrive/pnem1/train',
                                                 target_size = (224, 224),
                                                 batch_size = 64,
                                                 class_mode = 'binary')
test_set = test_datagen.flow_from_directory('/content/gdrive/MyDrive/pnem1/test',
                                            target_size = (224, 224),
                                            batch_size = 64,
                                            class_mode = 'binary')

Found 1510 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [22]:
labels = {value: key for key, value in training_set.class_indices.items()}

print("Label Mappings for classes present in the training and validation datasets\n")
for key, value in labels.items():
    print(f"{key} : {value}")

Label Mappings for classes present in the training and validation datasets

0 : NORMAL
1 : PNEUMONIA


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
es = EarlyStopping(patience = 2)
model_checkpoint_callback = ModelCheckpoint(
    filepath = 'model_checkpoints',
    save_weigths_only = True,
    monitor = 'val_accuracy',
    mode = 'max',
    save_best_only = True
)


from tensorflow.keras.applications.vgg16 import preprocess_input


history = model.fit(
    training_set,
    validation_data = test_set,
    epochs = 3,
    callbacks = [es,model_checkpoint_callback],
    use_multiprocessing = True,
    steps_per_epoch = 32,
    validation_steps = 32
)

In [24]:
test_loss, test_accuracy = model.evaluate(test_set)

10/10 [==============================] - 398s 40s/step - loss: 0.2930 - accuracy: 0.8814


In [30]:
from tensorflow.keras.models import save_model , load_model
save_model(model,'Pnem_CNN_vgg_16.h5')